# Einleitung

#### Überblick über den Titanic Datensatz

Der Titanic Datensatz ist ein berühmter Datensatz im Bereich des maschinellen Lernens und der Datenanalyse, der Daten über die Passagiere an Bord der Titanic und ihren Überlebensstatus enthält. Er besteht aus einem Trainingsdatensatz mit 891 Beobachtungen und einem Testdatensatz mit 418 Beobachtungen. Ziel ist es, das Überleben der Passagiere im Testdatensatz auf der Grundlage der Variablen im Trainingsdatensatz vorherzusagen.

# Importieren der Bibliotheken
Zunächst importieren wir die notwendigen Bibliotheken, die wir im Laufe dieses Notebooks verwenden werden:

In [ ]:
# Data Manipulation Libraries
import pandas as pd
import numpy as np

# Data Visualization Libraries
import seaborn as sns
import matplotlib.pyplot as plt

# Machine Learning Libraries
from sklearn.preprocessing import  StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score ,precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import  classification_report, confusion_matrix

# Machine Learning Models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC

# Laden des Datensatzes

In diesem Abschnitt werden wir den Titanic Datensatz in das Notebook laden. Der Datensatz ist in zwei CSV-Dateien gespeichert, eine für die Trainingsdaten und eine für die Testdaten. Wir verwenden die Pandas-Bibliothek, um die CSV-Dateien in Dataframes zu laden, die wir bearbeiten und untersuchen können. Nach dem Laden können wir damit beginnen, die Daten zu untersuchen und sie für die Modellierung mit maschinellem Lernen vorzubereiten.

In [ ]:
# Reading the train data
train_df = pd.read_csv("/kaggle/input/titanic/train.csv")

# Reading the test data
test = pd.read_csv("/kaggle/input/titanic/test.csv")

# Verstehen der Variablen

Bevor wir mit der Analyse des Titanic Datensatzes beginnen können, ist es wichtig zu verstehen, wofür die einzelnen Variablen stehen. Der Datensatz enthält die folgenden Variablen:

PassengerId: Eine eindeutige Kennung für jeden Passagier.

Survived: Ob der Passagier überlebt hat oder nicht (0 = No, 1 = Yes).

Pclass: Die Passagierklasse (1 = 1st class, 2 = 2nd class, 3 = 3rd class).

Name: Der Name des Passagiers.

Sex: Das Geschlecht des Fahrgastes.

Age: Das Alter des Fahrgastes in Jahren. Bei Kleinkindern werden Bruchteile angegeben.

SibSp: Die Anzahl der Geschwister/Ehegatten an Bord der Titanic.

Parch: Die Anzahl der Eltern/Kinder an Bord der Titanic.

Ticket: Die Ticketnummer des Passagiers.

Fare: Der vom Fahrgast bezahlte Fahrpreis.

Cabin: Die Kabinennummer des Passagiers (falls vorhanden).

Embarked: Der Hafen der Einschiffung (C = Cherbourg, Q = Queenstown, S = Southampton).

# Datenexploration und -aufbereitung

In [ ]:
# We use method head() to show the first 5 rows
train_df.head()

In [ ]:
test.head()

In [ ]:
print (test.shape)
print (train_df.shape)

Wie wir oben sehen, haben wir 11 Attribute und 418 Datensätze im Testdatensatz, aber wir haben 12 Attribute und 891 Datensätze im Trainingsdatensatz.


In [ ]:
train_df.describe()

Der obige Befehl `df.describe()` stellt statistische Eigenschaften in vertikaler Form dar.


In [ ]:
# Check for duplicated rows in train data
train_df.duplicated().sum()

In [ ]:
# Check for null values in train data
train_df.isna().sum()

In [ ]:
# Check for duplicated rows in test data
test.duplicated().sum()

In [ ]:
# Check for null values in test data
test.isna().sum()

In [ ]:
# The number of men who survived
train_df[train_df['Sex']=='male']['Survived'].sum()

In [ ]:
# The number of women who survived
train_df[train_df['Sex']=='female']['Survived'].sum()

Der folgende Code verwendet die Seaborn-Bibliothek, um Diagramme zu erstellen, die die Anzahl der überlebenden und nicht überlebenden Passagiere für jede der Spalten `Sex`, `Embarked`, `Pclass`, `SibSp` und `Parch` anzeigen.

In einer Schleife wird der Vorgang für jede angegebene Spalte in der Liste ['Sex', 'Embarked', 'Pclass', 'SibSp', 'Parch'] wiederholt.

Zum Zeichnen des Diagramms wird die Funktion "countplot" von Seaborn verwendet. Diese Funktion zeigt die Anzahl der Fälle in jeder Kategorie für die verschiedenen Werte der angegebenen Spalte (z. B. Geschlecht oder Ticketklasse) an, zusammen mit der charakteristischen Farbe für jede Kategorie (überlebt und nicht überlebt). Mit anderen Worten, die Anzahl der überlebenden und nicht überlebenden Passagiere wird für jeden unterschiedlichen Wert der angegebenen Spalte angezeigt.

In [ ]:

for column_name in ['Sex','Embarked','Pclass', 'SibSp', 'Parch']:
    print(column_name)
    sns.countplot(data=train_df, x=column_name, hue='Survived')
    plt.show()
    print("")

In [ ]:
sns.histplot(train_df['Age'])

Zum Zeichnen des Histogramms wird die Funktion `histplot` von Seaborn verwendet. Diese Funktion zeigt die Häufigkeitsverteilung der Spalte `Alter` an, d. h. die Anzahl der Passagiere in jeder Altersgruppe.

Anhand des Histogramms lassen sich Muster und Trends in der Altersverteilung der Fahrgäste im Datensatz `train_df` erkennen, z. B. die häufigste Altersgruppe oder das Auftreten von Ausreißern.

In [ ]:
# Calculate the mean age of male passengers in the 'train_df' dataset.
mean_male = train_df[train_df['Sex']=='male']['Age'].mean()
mean_male

In [ ]:
# Calculate the mean age of بثmale passengers in the 'train_df' dataset.
mean_female = train_df[train_df['Sex']=='female']['Age'].mean()
mean_female

Wie wir gesehen haben, gibt es einen Unterschied zwischen dem Mittelwert der Männer und dem Mittelwert der Frauen.

Im nachstehenden Code wird die Funktion `fillna` verwendet, um die fehlenden Werte in der Spalte `Age` für männliche und weibliche Fahrgäste mit den Werten `mean_male` bzw. `mean_female` zu füllen.

Dazu werden zunächst die Zeilen im `train_df`-Datensatz ausgewählt, in denen `Sex` `male` ist, indem die Bedingung `train_df['Sex']=='male'` verwendet wird. Dann werden für diese ausgewählten Zeilen die fehlenden Werte in der Spalte `Age` mit `mean_male` aufgefüllt.

In [ ]:
train_df.loc[train_df['Sex']=='male', 'Age'] = train_df[train_df['Sex']=='male']['Age'].fillna(value=mean_male)

In [ ]:
train_df.loc[train_df['Sex']=='female', 'Age'] = train_df[train_df['Sex']=='female']['Age'].fillna(value=mean_female)

In [ ]:
train_df.isna().sum()

Wie wir gesehen haben, gibt es keine Nullwerte in der Spalte `Age`.

In [ ]:
# Dropping some unimportant features
train_df.drop(['PassengerId','SibSp','Parch','Ticket','Cabin','Name'], axis=1, inplace= True)

In [ ]:
train_df.head()

In [ ]:
train_df.dropna(inplace=True)

Die Funktion `dropna` wird verwendet, um alle Zeilen im `train_df` Datensatz zu entfernen, die fehlende Werte (d. h. NaN-Werte) enthalten. Der Parameter `inplace=True` wird verwendet, um den Datensatz `train_df` direkt zu ändern, anstatt einen neuen geänderten Datensatz zurückzugeben.

In [ ]:
train_df.isna().sum()

Wie wir oben sehen, gibt es keine Nullwerte.

In [ ]:
train_df.replace({'female':0,'male':1},inplace=True)

Die Funktion `replace` wird verwendet, um die Werte `female` und `male` im Datensatz `train_df` durch 0 bzw. 1 zu ersetzen. Dazu wird ein Wörterbuch mit den Schlüsseln `female` und `male` und den entsprechenden Werten 0 und 1 an die Funktion `replace` übergeben. Der Parameter `inplace=True` wird verwendet, um den Datensatz `train_df` direkt zu ändern, anstatt einen neuen geänderten Datensatz zurückzugeben.

In [ ]:
train_df = pd.get_dummies(train_df,columns=['Embarked'],prefix='Embarked')

Die Funktion `get_dummies` von Pandas wird verwendet, um Dummy-Variablen für die Spalte `Embarked` im Datensatz `train_df` zu erstellen. Dazu werden der Datensatz `train_df` und die Spalte `Embarked` an den Parameter `columns` der Funktion `get_dummies` übergeben. Der Parameter `prefix` wird verwendet, um den Spaltennamen der resultierenden Dummy-Variablen das Präfix `Embarked_` hinzuzufügen.

Der resultierende Datensatz `train_df` enthält nun die neuen Spalten `Embarked_C`, `Embarked_Q` und `Embarked_S` mit binären Werten (0 oder 1), die angeben, ob ein Passagier vom entsprechenden Hafen aus eingeschifft wurde.

In [ ]:
train_df.head()

### Aufsplitten der Trainingsdaten

In [ ]:
# Split data into x and y
x = train_df.drop('Survived',axis =1)
y = train_df['Survived']
x

Dieser Code standardisiert die Werte der Spalten `Age` und `Fare` im `x` Datenframe mit Hilfe des StandardScaler aus der scikit-learn-Bibliothek.

In [ ]:
# Select numerical columns
num_cols = ['Age', 'Fare']

# Create scaler object
scaler = StandardScaler()

# Fit scaler on selected columns
scaler.fit(x[num_cols])

# Transform selected columns with scaler
x[num_cols] = scaler.transform(x[num_cols])

In [ ]:
# Split train data into train and test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)

# Modelbildung

In [ ]:
# Initialize the models
models = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'SVM': SVC(),
    'XGBClassifier':XGBClassifier(),
    'GradientBoostingClassifier':GradientBoostingClassifier(),
    'AdaBoostClassifier':AdaBoostClassifier()
    
}

# Train and evaluate each model using cross-validation
for name, model in models.items():
    scores = cross_val_score(model, x_train, y_train, cv=5, scoring='accuracy')
    print(f"{name} accuracy: {scores.mean():.3f} +/- {scores.std():.3f}")
    
    # Fit the model to the full training set and make predictions on the test set
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    
    # Evaluate the model on the test set
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    print(f"Accuracy: {acc:.3f}")
    print(f"Precision: {prec:.3f}")
    print(f"Recall: {rec:.3f}")
    print(f"F1-score: {f1:.3f}")
    print()

Nach dem Vergleich der Genauigkeit verschiedener Modelle stellten wir fest, dass das GradientBoostingClassifier-Modell die höchste Genauigkeit aufwies.

Ich werde sie also verwenden, um eine Vorhersage für die Testmenge zu treffen.

In [ ]:
gbc= GradientBoostingClassifier()
scores = cross_val_score(gbc, x_train, y_train, cv=5, scoring='accuracy')
print(f"{gbc} accuracy: {scores}")
    
# Fit the model to the full training set and make predictions on the test set
gbc.fit(x_train, y_train)
y_pred = gbc.predict(x_test)

# Evaluate the model on the test set
acc = accuracy_score(y_test, y_pred)
print (acc)

# Vorbereitung der Testdaten

In [ ]:
# Show the first 5 columns 
test.head()

In [ ]:
# Fill the null values in the age column
mean_male_test = test[test['Sex']=='male']['Age'].mean()
mean_female_test = test[test['Sex']=='female']['Age'].mean()
test.loc[test['Sex']=='male', 'Age'] = test[test['Sex']=='male']['Age'].fillna(value=mean_male_test)
test.loc[test['Sex']=='female', 'Age'] = test[test['Sex']=='female']['Age'].fillna(value=mean_female_test)

# Fill the null values in the Fare column
test['Fare'].fillna(test['Fare'].median(), inplace = True)

# Dropping some unimportant features
test.drop(['SibSp','Parch','Ticket','Cabin','Name'], axis=1, inplace= True)

In [ ]:
test.isna().sum()

Erneut keine Nullwerte zu entdecken.

In [ ]:
# Convert string values to numeric by replace the female and male by 0 and 1
test.replace({'female':0,'male':1},inplace=True)

In [ ]:
# Using one hot encoder to convert string values 
test = pd.get_dummies(test,columns=['Embarked'],prefix='Embarked')

In [ ]:
# Fit scaler on numeric columns
scaler.fit(test[num_cols])

# Transform numeric columns with scaler
test[num_cols] = scaler.transform(test[num_cols])

In [ ]:
test.head()

In [ ]:
# Store PassengerId column in a separate variable

PassengerId = test['PassengerId']

# Drop PassengerId column from the test set
test.drop('PassengerId',axis=1,inplace=True)

# Vorhersagen und Einreichung

In [ ]:
# Generate predictions for the test data using GradientBoostingClassifier 
test_pred = gbc.predict(test)

Erstellung einer Datei zum Hochladen mit der Spalte `PassengerId` und den vorhergesagten Überlebensergebnissen

In [ ]:
submission = pd.DataFrame({'PassengerId': PassengerId, 'Survived': test_pred})

In [ ]:
submission.to_csv('submission.csv', index=False)